![](https://europe-west1-atp-views-tracker.cloudfunctions.net/working-analytics?notebook=tutorials--aws-agentcore--agentcore-tutorial)

# AWS Bedrock AgentCore Tutorial: From Local Agent to Managed Agent

## Overview

This tutorial introduces AWS Bedrock AgentCore, a runtime framework designed to transform local AI agents into production-ready systems. We will walk through the process of creating a simple agent and then wrapping it with AgentCore to enable managed execution, request tracking, and standardized communication patterns.

By the end of this tutorial, you will understand how AgentCore serves as an infrastructure layer that sits between your agent logic and the production environment, handling the complexities of request management, execution control, and runtime orchestration.

### What You Will Learn

This tutorial covers three fundamental concepts. First, you will learn how to build a basic agent using standard Python. Second, you will understand how to wrap your agent with the AgentCore runtime using the Python SDK. Finally, you will see how to run your agent locally through a standardized HTTP interface that provides request tracking and structured responses.


## Understanding AgentCore Architecture

Before diving into implementation, it helps to understand what AgentCore does and why it matters. When you build an agent locally, it typically exists as a Python function or class that takes input and returns output. This works fine for development and testing, but production environments require additional infrastructure.

AgentCore provides this infrastructure layer. Think of it as a control tower for your agent. The control tower does not fly the plane, but it manages takeoffs, landings, and communications. Similarly, AgentCore does not change your agent's logic, but it manages how requests reach your agent, how responses are returned, and how the entire execution is tracked and monitored.

The framework introduces several key components. The runtime server handles incoming requests and routes them to your agent. The SDK provides a simple decorator pattern to convert your function into an HTTP service. Together, these components create a managed execution environment that transforms a local function into a production service.

### Why Use AgentCore

The primary value of AgentCore lies in separation of concerns. Your agent code focuses purely on processing messages and generating responses. AgentCore handles everything else: accepting HTTP requests, validating input, tracking execution time, managing errors, and formatting responses. This separation makes your code cleaner, easier to test, and simpler to maintain.

Additionally, AgentCore provides a foundation for production capabilities. Once your agent runs through the runtime, you can add tools, enable logging, implement access control, and deploy to any server without modifying your core agent logic. The infrastructure grows around your agent, not inside it.

## System Architecture Diagram

The following diagram illustrates how AgentCore transforms a local agent into a managed system. The flow begins with an HTTP request from a client, passes through the AgentCore runtime, reaches your agent for processing, and returns through the same path with added metadata.


#  ![AgentCore Architecture Diagram](assets/agentcore-diagram.png)
 




## Step 1: Environment Setup

We begin by installing the necessary dependencies and configuring our environment. AgentCore requires Python 3.10 or higher and depends on a few core libraries for HTTP handling and configuration management.

### Installing Dependencies

The following command installs the Bedrock AgentCore Runtime SDK along with supporting libraries. The `python-dotenv` package helps us manage API keys securely, while `requests` allows us to test our agent by making HTTP calls.

In [ ]:
!pip install bedrock-agentcore python-dotenv requests openai

### Configuring API Credentials

For this tutorial, we use OpenAI's API to power our agent's language understanding capabilities. The following code loads your API key from a `.env` file, which is a best practice for keeping sensitive credentials out of your code. You should create a `.env` file in your working directory with a line like `OPENAI_API_KEY=your-key-here`.

In [15]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

## Step 2: Building the Base Agent

Now we create our agent. This is a standard Python function that processes messages and returns responses. The agent is intentionally simple to keep the focus on understanding how AgentCore wraps and manages it.

### Agent Implementation

The `process_message` function demonstrates a basic agent that uses OpenAI's API to answer questions. This is just regular Python code with no special AgentCore-specific logic mixed in. Notice that we define the agent logic as a simple function that takes a message and returns a response.

In [ ]:
import openai

def process_message(message: str) -> str:
    """Process a message and return a response using OpenAI"""
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": message}],
        temperature=0
    )
    return response.choices[0].message.content

### Testing the Local Agent

Before integrating with AgentCore, we verify that our agent works correctly in isolation. This step is important because it confirms that any issues we encounter later are related to the runtime integration, not the agent logic itself.

In [17]:
result = process_message("What is AI?")
print("Agent Response:")
print(result)

Agent Response:
Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems. These processes include learning (the acquisition of information and rules for using it), reasoning (using rules to reach approximate or definite conclusions), and self-correction. AI can be categorized into two main types:

1. **Narrow AI (Weak AI)**: This type of AI is designed and trained for a specific task. Examples include virtual assistants like Siri and Alexa, recommendation systems, and image recognition software. Narrow AI operates under a limited set of constraints and is not capable of generalizing its knowledge to other tasks.

2. **General AI (Strong AI)**: This is a theoretical form of AI that possesses the ability to understand, learn, and apply intelligence across a wide range of tasks, similar to a human being. General AI would be able to perform any intellectual task that a human can do, but as of now, it remains largely a 

You should see a clear, informative explanation of artificial intelligence. At this point, we have a functioning agent, but it exists only as a local Python function. There is no server, no request handling, and no way for external systems to interact with it. This is where AgentCore comes in.

## Step 3: Integrating with AgentCore Runtime

This is the transformation step where we wrap our local agent with AgentCore's runtime infrastructure. The process involves creating a BedrockAgentCoreApp instance, decorating our entrypoint function, and understanding how the server works. Each part plays a specific role in creating a managed execution environment.

### Creating the AgentCore Application

The `BedrockAgentCoreApp` class is the core of the SDK. When you create an instance, it sets up all the infrastructure needed for handling HTTP requests, health checks, and response formatting. This single line of code initializes the entire runtime framework.

In [18]:
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

print("Bedrock AgentCore application created")

Bedrock AgentCore application created


### Agent Registration

Registration is how we connect our agent code to the runtime. The `@app.entrypoint` decorator tells AgentCore to call our function whenever a request arrives. The function signature is important: it receives a `payload` dictionary that contains the request data. By convention, AgentCore expects the user's message in a field called `prompt`.

Notice that inside the decorated function, we call our existing `process_message` function. This pattern gives you flexibility in how agents are created and managed. You could use dependency injection, implement caching, or add initialization logic here without changing the AgentCore integration.

In [19]:
@app.entrypoint
def invoke(payload: dict) -> dict:
    """
    Process user input and return a response.
    AgentCore will call this function for each HTTP request to /invocations
    """
    user_message = payload.get("prompt", "Hello")
    result = process_message(user_message)
    return {"result": result}

print("Agent registered with AgentCore")

Agent registered with AgentCore


### Starting the Runtime Server

Now we demonstrate how to start the actual server that will accept HTTP requests. The `app.run()` method starts an HTTP server on port 8080 by default. In a real deployment, you would save this code to a file (e.g., `agent.py`) and run it with `python agent.py`.

Note: Running `app.run()` directly in a notebook will block execution. For production, you would run this in a separate process or use the AgentCore starter toolkit for deployment.

In [20]:
# In a real scenario, you would run this in a separate file:
# if __name__ == "__main__":
#     app.run()

# For notebook demonstration, we show the complete agent code:
complete_agent_code = '''
import os
import openai
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from dotenv import load_dotenv

load_dotenv()

app = BedrockAgentCoreApp()

def process_message(message: str) -> str:
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": message}],
        temperature=0
    )
    return response.choices[0].message.content

@app.entrypoint
def invoke(payload: dict) -> dict:
    user_message = payload.get("prompt", "Hello")
    result = process_message(user_message)
    return {"result": result}

if __name__ == "__main__":
    app.run()  # Server starts on http://localhost:8080
'''

print("Complete agent code structure:")
print(complete_agent_code)
print("\nKey endpoints when running:")
print("  POST http://localhost:8080/invocations - Send requests to your agent")
print("  GET http://localhost:8080/ping - Health check endpoint")

Complete agent code structure:

import os
import openai
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from dotenv import load_dotenv

load_dotenv()

app = BedrockAgentCoreApp()

def process_message(message: str) -> str:
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": message}],
        temperature=0
    )
    return response.choices[0].message.content

@app.entrypoint
def invoke(payload: dict) -> dict:
    user_message = payload.get("prompt", "Hello")
    result = process_message(user_message)
    return {"result": result}

if __name__ == "__main__":
    app.run()  # Server starts on http://localhost:8080


Key endpoints when running:
  POST http://localhost:8080/invocations - Send requests to your agent
  GET http://localhost:8080/ping - Health check endpoint


At this point, your agent is no longer just a Python function. When you run the code as a standalone script, it becomes a service that accepts HTTP requests, processes them through your agent logic, and returns structured responses. The AgentCore SDK has automatically added health check endpoints, request validation, and response formatting.

## Step 4: Interacting with the Managed Agent

With the runtime server active (when running outside this notebook), we can interact with our agent through HTTP requests. This represents a fundamental shift from direct function calls to network-based communication. The agent logic has not changed, but the way we access it has been transformed.

### Making Your First Request

We use the `requests` library to send an HTTP POST request to our running AgentCore server. The request goes to the `/invocations` endpoint with a JSON payload containing a `prompt` field. This is the standardized format that AgentCore expects. The runtime receives this request, extracts the payload, calls our decorated entrypoint function, and packages the response.

In [21]:
# This code would work when the server is running in a separate process:
import requests
import json

# Example HTTP request structure:
request_example = {
    "url": "http://localhost:8080/invocations",
    "method": "POST",
    "headers": {"Content-Type": "application/json"},
    "body": {"prompt": "What is AI?"}
}

print("Example HTTP request to AgentCore:")
print(json.dumps(request_example, indent=2))
print("\nExample using curl:")
print("curl -X POST http://localhost:8080/invocations \\")
print("  -H 'Content-Type: application/json' \\")
print("  -d '{\"prompt\": \"What is AI?\"}'")

# If the server were running, you would make the actual request:
# response = requests.post(
#     "http://localhost:8080/invocations",
#     json={"prompt": "What is AI?"}
# )
# result = response.json()
# print(json.dumps(result, indent=2))

Example HTTP request to AgentCore:
{
  "url": "http://localhost:8080/invocations",
  "method": "POST",
  "headers": {
    "Content-Type": "application/json"
  },
  "body": {
    "prompt": "What is AI?"
  }
}

Example using curl:
curl -X POST http://localhost:8080/invocations \
  -H 'Content-Type: application/json' \
  -d '{"prompt": "What is AI?"}'


### Understanding the Response Structure

The response you receive from AgentCore follows the structure you defined in your entrypoint function. In our case, we return `{"result": result}`, so the HTTP response will contain this structure. This makes it easy to build systems that consume your agent's output, as the format is predictable and consistent.

In [22]:
# Expected response structure based on our entrypoint function:
expected_response = {
    "result": "AI (Artificial Intelligence) refers to computer systems that can perform tasks typically requiring human intelligence..."
}

print("Expected response structure:")
print(json.dumps(expected_response, indent=2))

Expected response structure:
{
  "result": "AI (Artificial Intelligence) refers to computer systems that can perform tasks typically requiring human intelligence..."
}


### Testing with Different Inputs

To better understand how the system works, here is another example request with a different question. Each request is handled independently through the same entrypoint function.

In [23]:
# Another example request:
print("Example request for machine learning explanation:")
print("curl -X POST http://localhost:8080/invocations \\")
print("  -H 'Content-Type: application/json' \\")
print("  -d '{\"prompt\": \"Explain machine learning in simple terms\"}'")

# Expected response:
print("\nExpected response:")
print('{"result": "Machine learning is..."}')

Example request for machine learning explanation:
curl -X POST http://localhost:8080/invocations \
  -H 'Content-Type: application/json' \
  -d '{"prompt": "Explain machine learning in simple terms"}'

Expected response:
{"result": "Machine learning is..."}


## Comparing Local and Managed Execution

It is worth pausing to consider what has changed and what has stayed the same. Your agent's core logic—the `process_message` function that calls the OpenAI API and returns a response—remains completely unchanged. You did not modify the agent code to work with AgentCore.

What changed is how you interact with the agent. Before AgentCore, you called a function directly:

```python
result = process_message("What is AI?")
print(result)
```

After AgentCore, you send an HTTP request:

```python
response = requests.post(
    "http://localhost:8080/invocations",
    json={"prompt": "What is AI?"}
)
result = response.json()
```

This change brings significant benefits. Your agent is now network-accessible, meaning it can be called from any application that speaks HTTP. You can deploy it to AWS infrastructure using the AgentCore starter toolkit. You can add memory capabilities, implement access control, and scale to handle production traffic. Most importantly, you have built a foundation that supports all these enhancements without modifying your core agent logic.

This separation of concerns—agent logic versus infrastructure—is the key insight of AgentCore. Your agent focuses on being good at its task. AgentCore focuses on making that agent accessible, manageable, and production-ready.

## Summary

In this tutorial, you learned the fundamental pattern of working with AWS Bedrock AgentCore. You created a simple agent, wrapped it with the AgentCore runtime SDK using the `@app.entrypoint` decorator, and understood how to interact with it through a managed HTTP interface. The agent code itself remained clean and focused, while AgentCore handled the infrastructure concerns.

This foundation opens up several paths for enhancement. You can add tools that allow your agent to search the web, query databases, or call external APIs. You can enable detailed logging to see exactly what your agent does at each step. You can implement identity and access control to restrict who can use certain capabilities. You can deploy your agent to AWS using the AgentCore starter toolkit where it handles real user requests.

All of these enhancements build on the same pattern you learned here: your agent focuses on its core task, while AgentCore manages the infrastructure around it. This separation makes your code easier to test, maintain, and evolve as requirements change.



